In [ ]:
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.vec_env import DummyVecEnv
from sb3_contrib import RecurrentPPO # <--- La joya de la corona
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl import config_tickers

# ==============================================================================
# 1. BLOQUE DE DATOS: INGENIERÍA HÍBRIDA
# ==============================================================================
def prepare_custom_data():
    print("1. Descargando Precios + Macro de Yahoo...")
    # A. Tickers del Dow 30 (Sin WBA)
    dow_tickers = [t for t in config_tickers.DOW_30_TICKER if t != 'WBA']
    df_prices = YahooDownloader(start_date='2018-01-01', end_date='2023-01-01', ticker_list=dow_tickers).fetch_data()

    # B. Datos Macro (VIX, Bonos, Oro)
    macro_tickers = ['^VIX', '^TNX', 'GC=F']
    df_macro = YahooDownloader(start_date='2018-01-01', end_date='2023-01-01', ticker_list=macro_tickers).fetch_data()
    
    # Procesar Macro: Pivotar para tener columnas [date, ^VIX, ^TNX, GC=F]
    df_macro = df_macro.pivot(index='date', columns='tic', values='close').reset_index()
    # Rellenar fines de semana/festivos de bonos
    df_macro = df_macro.ffill()

    print("2. Calculando Indicadores Técnicos...")
    fe = FeatureEngineer(use_technical_indicator=True,
                         tech_indicator_list=['macd', 'rsi_30'], # Manténlo simple para LSTM
                         use_turbulence=False,
                         user_defined_feature=False)
    df_prices = fe.preprocess_data(df_prices)

    print("3. Fusionando con Datos Fundamentales (Simulados/Capital IQ)...")
    # AQUI ENTRA TU EXCEL DE CAPITAL IQ
    # df_capiq = pd.read_excel("tus_datos_capiq.xlsx") 
    # df_prices = df_prices.merge(df_capiq, on=['date', 'tic'], how='left')
    # df_prices['eps_norm'] = df_prices.groupby('tic')['eps_norm'].ffill()
    # df_prices['pe_ratio'] = df_prices['close'] / df_prices['eps_norm']

    print("4. Uniendo Macro a cada Acción...")
    # Unimos el VIX y Bonos a cada fila de cada acción
    df_final = df_prices.merge(df_macro, on='date', how='left')
    
    # Ordenar y rellenar cualquier hueco final
    df_final = df_final.sort_values(['date', 'tic']).ffill().dropna()
    
    # Generar Covarianzas (Necesario para Portfolio Env)
    df_final = add_covariance_matrix(df_final) # (Función auxiliar que ya tienes)
    
    # Indexar por ID numérico de día
    df_final.index = df_final.date.factorize()[0]
    
    return df_final, len(dow_tickers)

def add_covariance_matrix(df, lookback=252):
    # (Tu código de covarianza va aquí)
    # ... Pega tu bucle de covarianza aquí ...
    # Para el ejemplo, retornamos el df tal cual si ya lo tienes
    return df